In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, smart_resize
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import Adam
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.models import load_model
from keras.metrics import AUC
from tqdm.auto import tqdm
sns.set_style('darkgrid')
pd.set_option("display.max_columns", None)
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from glob import glob
from collections import defaultdict
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Conv2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121, EfficientNetB0, Xception
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
train_df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train_path = "../input/plant-pathology-2021-fgvc8/train_images"

In [ ]:
mlb = MultiLabelBinarizer().fit(train_df.labels.apply(lambda x : x.split()))
labels = pd.DataFrame(mlb.transform(train_df.labels.apply(lambda x : x.split())), columns = mlb.classes_)

labels = pd.concat([train_df['image'], labels], axis=1)
labels.head()

In [ ]:
model = load_model('../input/plant2021-efficientnet-training/effnetb0image512.h5')

In [ ]:
submissions = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submissions.head()

In [ ]:
image_test_datagen = ImageDataGenerator(
    rescale=1/255.
)

In [ ]:
test_path = "../input/plant-pathology-2021-fgvc8/test_images"
IMAGE = (256, 256)
BATCH_SIZE = 64
test_generator = image_test_datagen.flow_from_dataframe(
    submissions,
    directory=test_path,
    x_col="image",
    y_col="labels",
    target_size = IMAGE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42,
    subset=None
)

In [ ]:
predicts = model.predict(test_generator)
predicts

In [ ]:
verdict = (predicts>0.25)

In [ ]:
for x in verdict:
    if x[2]:
        for i in range(len(x)):
            x[i]=False
        x[2]=True

In [ ]:
verdict

In [ ]:
label = labels.columns.tolist()[1:]
label

In [ ]:
pred_lists = []
for i in range(verdict.shape[0]):
    tmp = []
    for j, c in enumerate(label):
        if verdict[i, j]:
            tmp.append(c)
    pred_lists.append(tmp)

pred_lists = [' '.join(t) for t in pred_lists]
pred_lists

In [ ]:
submissions['labels'] = np.array(pred_lists)

In [ ]:
submissions.to_csv('submission.csv', index=False)